In [1]:
#KEY ESTIMATION NEURAL NETWORK

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.conv2 = nn.Conv2d(8, 8, 5)
        self.conv3 = nn.Conv2d(8, 8, 5)
        self.conv4 = nn.Conv2d(8, 8, 5)
        self.conv5 = nn.Conv2d(8, 8, 5)
        

        
        x = torch.randn(3,105,600).view(-1,3,105,600) # giving a random input to find the input of the first linear layer 
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 1024)
        self.fc2 = nn.Linear(1024, 64)
        self.fc3 = nn.Linear(64, 12)
        self.dropout = nn.Dropout(0.5)
        
    def convs(self, x):
        
        x = F.elu(self.conv1(x))        
        x = F.elu(self.conv2(x))        
        x = F.elu(self.conv3(x))        
        x = F.elu(self.conv4(x))        
        x = F.avg_pool2d(F.elu(self.conv5(x)), (2,2))

                
        if self._to_linear is None: #we have to define the input of the first linear layer 
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
        
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.elu(self.fc1(x))
        x = self.dropout(x)
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        return x
    
if torch.cuda.is_available(): # set the variables at GPU 
    device = torch.device("cuda:0")
    k_Net = Net().to(device)


print(k_Net)


Net(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv5): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=97440, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=12, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [2]:
#CHORD ESTIMATION NEURAL NETWORK (binary classification)"

import torch
import torch.nn as nn
import torch.nn.functional as F


class chordNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,16,5)
        self.conv2 = nn.Conv2d(16,16,5)
        
        x = torch.randn(105,600).view(-1,1,105,600) # giving a random input to find the input of the first linear layer 
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 8)
        self.fc2 = nn.Linear(8, 2)
        self.dropout = nn.Dropout(0.5)        
        
        
    def convs(self, x):
        x = F.elu(self.conv1(x))
        x = F.avg_pool2d(F.elu(self.conv2(x)), (2,2))
        
        
        if self._to_linear is None: # we have to define the input of the first linear layer 
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    
    def forward(self, x):
        
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.elu(self.fc1(x))
        x = self.dropout(x)
        x = F.elu(self.fc2(x))
        
        return x
        
        
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    c_Net = chordNet().to(device)


print(c_Net)

chordNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=227328, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [3]:
#functions to convert WAV to spectrogram 

import librosa
import numpy as np
import skimage.io
import os
import numpy


# scale the values from 0 to 1 

def scale_minmax(X, min=0.0, max=1.0): 
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

# making the spectrogram

def spectrogram_image(y, sr, out, hop_length, n_mels): 
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = numpy.log(mels + 1e-9) # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = numpy.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(out, img)
    


In [ ]:
# EKTELESIMO PAROUSIASHS !!!

import librosa
import numpy as np
import skimage.io
import os 
import cv2
import torch
import os
import pydub
from pydub import AudioSegment
from PIL import Image 
import numpy as np
import matplotlib.pyplot as plt


song = "nothing"
keys = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']


C_model = c_Net
C_model.load_state_dict(torch.load("C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/saved_nns/best-chord-network-model-parameters.pt"))
C_model.eval()

K_model = k_Net
K_model.load_state_dict(torch.load("C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/saved_nns/best-key-network-model-parameters.pt"))
K_model.eval()



path = "C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/final_dataset_mp3"
os.chdir(path)
mp3_files = os.listdir()

while (song != "0"):
    found = 0
    
    print("\nSelect a song from the list:")
    
    if (song == "nothing"): #print all the songs from our list 
        for i,file in enumerate(mp3_files):
            name, ext = os.path.splitext(file)
            print(f"{i+1} {name}")
            if(i==41):
                print("\n")
    
    song = input("") # read from keyboard 
    
    for file in mp3_files : #  search for the song
        name, ext = os.path.splitext(file)
        
        if (name == song):
            my_mp3 = file
            found = 1
            break;


    #if name of the song exists 
    if (found == 1):
        
        #converting to WAV file 
        
        print("\nConverting mp3 to WAV...")
        pydub.AudioSegment.converter = r"C:\ffmpeg\bin\ffmpeg.exe"
        mp3_sound = AudioSegment.from_mp3(my_mp3)
        #rename them using the old name + ".wav"
        mp3_sound.export("C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/test_dataset_wav/{0}.wav".format(name), format="wav")
        
        
        print("-- mp3 to WAV convertion accomplished --")
        
        #converting WAV to spectrogram
        print("\nConverting WAV to spectrogram...")
        hop_length = 4096 # number of samples per time-step in spectrogram
        n_mels = 105 # number of bins in spectrogram. Height of image
        time_steps = 599 # number of time-steps. Width of image

        # extract a fixed length window
        start_sample = 0 # starting at beginning
        length_samples = time_steps*hop_length


        # load audio. Using librosa
        out = 'C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/spectro_file/{0}.png'.format(name) #set the output folder for spectrogram
        
        wav_file = 'C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/test_dataset_wav/{0}.wav'.format(name)
        
        y, sr = librosa.load(wav_file, sr=44100)
       
        window = y[start_sample:start_sample+length_samples]
        # convert to PNG
        spectrogram_image(window, sr=sr, out=out, hop_length=hop_length, n_mels=n_mels)
       
        print('~~ WAV to spectro convertion accomplished ~~')
        
        
        #print spectrogram

        img = Image.open('C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/spectro_file/{0}.png'.format(name))
        
        print("\nSpectrogram :")
        plt.imshow(img)
        plt.show()

        #predicting the chord 
        
        my_spectro = 'C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/spectro_file/{0}.png'.format(name)
        chord_spect = cv2.imread(my_spectro,cv2.IMREAD_GRAYSCALE)
        value = np.array(chord_spect)/255

        T_value = torch.from_numpy(value)  # convert numpy.ndarray to Tensor to feed it in the neural network
        c_value = T_value.view(-1,1,105,600)
        c_value = c_value.float()
        c_value = c_value.to(device)

        chord_output = C_model(c_value)
        
        #predicting the key 
        
        key_spectro = cv2.imread(my_spectro)
        value = np.array(key_spectro)/255
        
        T_value = torch.from_numpy(value)  # convert numpy.ndarray to Tensor to feed it in the neural network 
        k_value = T_value.view(-1,3,105,600)
        k_value = k_value.float()
        k_value = k_value.to(device)
        
        key_output = K_model(k_value)
        
        found=0
    elif (song != "0"):
        print("\n**Song not found please try again**")
        continue;
    else:
        continue;
    
    
    
    _,chord_predict = torch.max(chord_output.data,1)
    _,key_predict = torch.max(key_output.data,1)
    
    
    
    if (chord_predict == 0) : # predicted minor
        print (f"\nThe predicted key of {song} is: {keys[key_predict]} minor\n")
        
    else: # predicted major
        print (f"\nThe predicted key of {song} is: {keys[key_predict]} major\n")

    
    ann_file = open("C:/outs/24Annotations.txt") #open the annotation file to check the correct key of the song 
    while True:
        line = ann_file.readline()
        ann_name, key, chord = line.split(' ', 2)
        ann_name = ann_name.strip()
        key = key.strip()
        chord = chord.strip()
        
        if (ann_name == song):
            break;
            
            
    print(f"The correct answer of {ann_name} is:\n\n{key} {chord}\n")
    print("--------------------------------------------------------\n")
    
    #delete wav and spectro files
    os.remove('C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/test_dataset_wav/{0}.wav'.format(name))
    os.remove('C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/spectro_file/{0}.png'.format(name))
    
print("\n\nΕΥΧΑΡΙΣΤΟΥΜΕ ΠΟΛΥ ΓΙΑ ΤΗΝ ΠΡΟΣΟΧΗ ΣΑΣ !!!")

